In [45]:
import linkedin.darwinfilemanager

In [46]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import itertools
from scipy.interpolate import make_interp_spline
from matplotlib.ticker import PercentFormatter
from matplotlib.pyplot import plot, savefig

# Score Extraction

In [47]:
%%capture
%reload_ext linkedin.lisql
%config SqlMagic.autocommit=False
%manage_trino holdem
%sql SET SESSION li_authorization_user = 'gtme';

## Q2FY25 DATA:

In [48]:
%%sql scores <<
WITH base AS (
    SELECT
        c.callid,
        array_agg(DISTINCT z.msft_employee ORDER BY z.msft_employee DESC) AS speaker_types
         
    FROM u_gtme.lms_bpscore_combined AS c
     
    LEFT JOIN u_superstore.gong_calls_detail_2 AS z
        ON c.callid = z.callid
         
    WHERE z.level_03_manager_name != 'Lekha Doshi' --no vendor CSM/AE/other calls
        AND z.duration > 300 -- calls more than 5 minutes
    GROUP BY c.callid
         
),
 
relevant_calls AS (

    SELECT DISTINCT callid
    FROM base
    WHERE array_join(speaker_types, ',') LIKE '%MSFT%CLIENT%' -- calls with atleat 1 customer & rep on call
    
),

authorised_countries AS (
 SELECT *
 FROM (VALUES
  ('United States'), 
  ('Canada'),
  ('United Kingdom'),
  ('Ireland'),
  ('Spain'),
  ('Italy'), 
  ('Israel'),
  ('Brazil'),
  ('Bulgaria'),
  ('Netherlands'),
  ('Sweden'),
  ('Denmark'),
  ('Finland'),
  ('Norway'),
  -- ('France'),
  ('Mexico'),
  ('Chile'),
  ('Argentina'),
  ('Colombia'),
  ('Peru'),
  ('Australia'),
  ('New Zealand'),
  ('India'),
  ('Singapore')
 ) AS t (country)
),
all_data AS (
SELECT 
 
  lms_bpscore_combined.callid,
  
  calls.crm_country,
  -- calls.emailaddress,
 
  users.level_03_manager_name,
  users.level_04_manager_name,
  -- users.level_05_manager_name,
  -- users.business_unit,
  users.role, 
  -- users.d365_user_role_type,
  users.job_title,
  users.job_family,
  users.work_location_country_name AS rep_country,
  users.region AS rep_region,
  users.rep_segment AS rep_segment,
  calls.calldate,
  calls.title,
  calls.duration,
  calls.crm_country,
  users.linkedin_emails[1] AS emailaddress,
  
  lms_bpscore_combined.research_score,
  lms_bpscore_combined.highimpactquestions_score,
  lms_bpscore_combined.examinechallenges_score,
  lms_bpscore_combined.present_impact_story,
  lms_bpscore_combined.share_key_insights,
  lms_bpscore_combined.align_unique_differentiators,
  lms_bpscore_combined.multithreading_score,
  lms_bpscore_combined.deliver_compelling_proposals,
  lms_bpscore_combined.prove_roi,
  lms_bpscore_combined.handle_objections,
  lms_bpscore_combined.navigate_decisionmaking_processes,
  lms_bpscore_combined.set_purposeful_milestones,
  lms_bpscore_combined.establish_alignment_accountability
  
 FROM u_gtme.lms_bpscore_combined
  
 LEFT JOIN u_superstore.gong_calls_detail_2 AS calls
  ON lms_bpscore_combined.callid = calls.callid
  AND calls.call_row_cnt = 1
  
 LEFT JOIN u_superstore.org_and_user_dimensions_final AS users
  ON CONCAT(SUBSTRING(CAST(calls.calldate AS VARCHAR), 1, 8), '01') = users.month_begin_date
  AND users.current_workday_id = calls.employee_id
  
 WHERE lms_bpscore_combined.callid IN (SELECT callid FROM relevant_calls)
  AND calls.crm_country IN (SELECT country FROM authorised_countries) -- crm_country is None
  AND users.work_location_country_name IN (SELECT country FROM authorised_countries)
  AND calls.callid NOT IN (
   '7975914294131661578',
   '6578777980626122142'
  )
  
)


SELECT * from all_data;

## Q3 DATA:

In [ ]:
%%sql scores <<
SELECT 
    t5.*,
    t6.job_title, 
	t6.role, 
	t6.rep_segment,
	t6.rep_vertical, 
	t6.work_location_country_name
FROM u_gtme.lms_bpscore_combined t5
    --u_gtme.LMS_BPskill_combined t5
LEFT JOIN u_superstore.org_and_user_dimensions_final t6 
        ON t5.employee_id  = t6.current_workday_id 
            AND CONCAT(SUBSTRING(CAST(t5.calldate AS VARCHAR), 1, 8), '01') = t6.month_begin_date
           
;

In [95]:
df = scores.copy()
df['callid'].nunique()

In [96]:
df.head()

In [73]:
df.columns

In [21]:
skill_columns = [
    'research_score', 'highimpactquestions_score', 'examinechallenges_score',
    'present_impact_story', 'share_key_insights', 'align_unique_differentiators',
    'deliver_compelling_proposals', 'prove_roi', 'handle_objections',
    'navigate_decisionmaking_processes', 'set_purposeful_milestones',
    'establish_alignment_accountability'
]

In [22]:
missing_values = df[skill_columns].isnull().sum()
print("Missing values in skill score columns:")
print(missing_values)

In [23]:
cleaned_data = df.dropna(subset=skill_columns)
print(cleaned_data.shape)

In [24]:
missing_values = cleaned_data[skill_columns].isnull().sum()
print("Missing values in skill score columns:")
print(missing_values)

In [25]:
correlation_matrix = cleaned_data[skill_columns].corr()
print(correlation_matrix.shape)

### 1. Correlation Analysis visualizes relationships among skill scores.

In [26]:
plt.figure(figsize=(12, 12))

In [27]:
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm')

In [28]:
sns.heatmap(correlation_matrix, fmt='.2f', cmap='coolwarm')

In [29]:
plt.title("Correlation Matrix of Skill Scores")
plt.show()


### 2. Group Analysis calculates average skill scores segmented by roles, segments, verticals, and sales regions.

In [54]:
--#unique_values = df['role'].unique()
unique_values = df['job_title'].unique()
print(unique_values)

In [75]:
#--unique_values = df[sales_region'].unique()
unique_values = cleaned_data['rep_segment'].unique()
print(unique_values)


In [91]:

unique_values = cleaned_data['rep_vertical'].unique()
print(unique_values)

In [76]:

unique_values = cleaned_data['sales_region'].unique()
print(unique_values)

In [88]:
# Define mappings for regions and segments
segment_mapping = {
    'ENTERPRISE': 'ESG',
    'UNKNOWN': 'unknown/other',
    'OTHER': 'unknown/other'
}
region_mapping = {
    'EMEA': 'EMEAL',
    'LATAM': 'EMEAL',
    'None' : 'APAC'
}

job_title_mapping = {
'Client Solutions Manager 3': 'CSM',
'LMS ESG Account Executive 3' : 'AE',
'LMS OSO POR Account Executive 3' : 'SPOC',
'LMS OSO New Biz Account Executive 3' : 'AE',
 'LMS ESG Account Executive 4' : 'AE',
'LMS OSO POR Account Executive 2' : 'SPOC',
 'Account Executive - Marketing Solutions'  : 'AE',
'Client Solutions Manager 2': 'CSM',
 'LMS OSO New Biz Account Executive 4' : 'AE',
'Manager 1, LMS OSO New Biz AE' : 'AE',
 'Client Solutions Manager 4' : 'CSM',
'Manager 3, LMS OSO POR AE' : 'SPOC',
 'LMS ESG Account Executive 5' : 'AE',
'LMS Strategic Account Executive 4' : 'AE',
 'Client Solutions Manager, LinkedIn Marketing Solutions': 'CSM',
 'LMS OSO Account Executive 2' : 'AE',
'LMS OSO Account Executive 1' : 'AE',
 'LMS OSO Account Executive 3' : 'AE',
 'Account Executive, LinkedIn Marketing Solutions' : 'AE',
 'LMS OSO POR Account Executive 1' : 'SPOC',
'Manager 2, LMS OSO POR AE' : 'SPOC',
 'Manager 3, LMS OSO New Biz AE'  : 'AE',
'LMS OSO POR Account Executive 4' : 'SPOC',
 'LMS OSO New Biz Account Executive 2' : 'AE',
'LMS ESG Account Executive 2' : 'AE',
 'Manager 3, LMS ESG AE' : 'AE',
'LMS Strategic Account Director' : 'AD',
 'Manager 2, LMS ESG AE'  : 'AE',
'Vendor Partner Lead 3' : 'Vendor Partner Lead',
 'Key Client Solutions Manager': 'Key CSM'
}    


data = cleaned_data.copy()



In [89]:
# Apply mappings using .loc[]
data.loc[:, 'sales_region'] = data['sales_region'].replace(region_mapping)
data.loc[:, 'rep_segment'] = data['rep_segment'].replace(segment_mapping)
data.loc[:, 'job_title'] = data['job_title'].replace(job_title_mapping)

# Verify changes (optional)
print(data[['sales_region', 'rep_segment','job_title']].drop_duplicates())

In [98]:
group_columns = ['rep_segment','rep_vertical', 'sales_region','job_title']
skill_means_by_group = data.groupby(group_columns)[skill_columns].mean().round(1)

skill_means_by_group

In [104]:

skill_means_by_group.loc['ESG']

### 3. Performance Comparison: Identify top and bottom performers by aggregating normalized skill scores (z-scores).

In [1]:

for skill in skill_columns:
    data[f'{skill}_zscore'] = (data[skill] - data[skill].mean()) / data[skill].std()

# Summing z-scores to create an aggregate performance score
data['aggregate_performance'] = data[[f'{col}_zscore' for col in skill_columns]].sum(axis=1)

# Analyzing top and bottom 10 performers
top_performers = data.nlargest(10, 'aggregate_performance')
bottom_performers = data.nsmallest(10, 'aggregate_performance')





In [ ]:
top_performers

In [ ]:
bottom_performers

### 4. ANOVA Analysis checks significant differences in skill scores across roles.



In [9]:
# 4. Analysis of Variance (ANOVA): Check if skill scores differ significantly by group
from scipy.stats import f_oneway

anova_results = {}
for skill in skill_columns:
    anova_results[skill] = f_oneway(
        *[group[skill].dropna().values for _, group in data.groupby('role')]
    )

anova_results_df = pd.DataFrame({
    'Skill': skill_columns,
    'F-Statistic': [anova_results[skill].statistic for skill in skill_columns],
    'p-Value': [anova_results[skill].pvalue for skill in skill_columns]
})

# Export ANOVA results to CSV
#anova_results_df.to_csv("anova_results.csv", index=False)


### 5. Missing Data Analysis identifies missing or incomplete values within the dataset.

In [ ]:
# 5. Identify Missing or Incomplete Data for Skills by Group Dimensions
missing_value_summary = df[group_columns + skill_columns].isnull().sum()

# Export missing value summary
missing_value_summary.to_csv("missing_value_summary.csv")

# Optional: Visualize performance by group using boxplots
for skill in skill_columns:
    plt.figure(figsize=(12, 6))
    sns.boxplot(x='role', y=skill, data=df)
    plt.title(f"{skill} Distribution by Role")
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
['callid', 'fiscal_quarter', 'calldate', 'title', 'duration',
       'employee_id', 'emailaddress', 'sales_region', 'crm_country',
       'research_score', 'research_comments', 'highimpactquestions_score',
       'highimpactquestions_comments', 'examinechallenges_score', 'examinechallenges_comments', 'present_impact_story', 'present_impact_story_comments', 'share_key_insights',
       'share_key_insights_comments', 'align_unique_differentiators', 'align_unique_differentiators_comments', 'deliver_compelling_proposals', 'deliver_compelling_proposals_comments',
       'deal_negotiation', 'deal_negotiation_comments', 'handle_objections','handle_objections_comments', 'navigate_decisionmaking_processes', 'navigate_decisionmaking_processes_comments',
       'set_purposeful_milestones', 'set_purposeful_milestones_comments','establish_alignment_accountability','establish_alignment_accountability_comments',
       'job_title', 'role', 'rep_segment', 'rep_vertical', 'work_location_country_name']

### 6. Best In Class

## Q2:

In [97]:
# Define base columns to keep (informative only)
base_cols = [
    'callid', 'calldate', 'title', 'duration', 'emailaddress',
    'rep_segment', 'role', 'job_title'
]

# Define skill-comment pairs to melt
skill_scores = {
    'research': ('research_score'),
    'high_impact_questions': ('highimpactquestions_score'),
    'examine_challenges': ('examinechallenges_score'),
    
    'present_the_impact_story': ('present_impact_story'),
    'share_key_insights': ('share_key_insights'),
    'align_unique_differentiators': ('align_unique_differentiators'),

    'deliver_compelling_proposals': ('deliver_compelling_proposals'),
    'prove_roi': ('prove_roi'),
    'handle_objections': ('handle_objections'),
    
    'navigate_decision_making_processes': ('navigate_decisionmaking_processes'),
    'set_purposeful_milestones': ('set_purposeful_milestones'),
    'establish_alignment_accountability': ('establish_alignment_accountability')
}

# Melt each pair into long format
long_df = pd.DataFrame()
for skill, (score_col) in skill_scores.items():
    temp = df[base_cols].copy()
    temp['behavior'] = skill
    temp['score'] = df[score_col]
    long_df = pd.concat([long_df, temp], ignore_index=True)

# Preview the final melted dataframe
long_df.head()


## Q3

In [55]:
# Define base columns to keep (informative only)
base_cols = [
    'callid', 'calldate', 'title', 'duration', 'employee_id', 'emailaddress',
    'work_location_country_name', 'sales_region', 'rep_segment', 'rep_vertical', 'role', 'job_title'
]

# Define skill-comment pairs to melt
skill_comment_pairs = {
    'research': ('research_score', 'research_comments'),
    'high_impact_questions': ('highimpactquestions_score', 'highimpactquestions_comments'),
    'examine_challenges': ('examinechallenges_score', 'examinechallenges_comments'),
    
    'present_the_impact_story': ('present_impact_story', 'present_impact_story_comments'),
    'share_key_insights': ('share_key_insights', 'share_key_insights_comments'),
    'align_unique_differentiators': ('align_unique_differentiators', 'align_unique_differentiators_comments'),

    'deliver_compelling_proposals': ('deliver_compelling_proposals', 'deliver_compelling_proposals_comments'),
    'negotiating_deal_structure': ('deal_negotiation', 'deal_negotiation_comments'),
    'handle_objections': ('handle_objections', 'handle_objections_comments'),
    
    'navigate_decision_making_processes': ('navigate_decisionmaking_processes', 'navigate_decisionmaking_processes_comments'),
    'set_purposeful_milestones': ('set_purposeful_milestones', 'set_purposeful_milestones_comments'),
    'establish_alignment_accountability': ('establish_alignment_accountability', 'establish_alignment_accountability_comments')
}

# Melt each pair into long format
long_df = pd.DataFrame()
for skill, (score_col, comment_col) in skill_comment_pairs.items():
    temp = df[base_cols].copy()
    temp['behavior'] = skill
    temp['score'] = df[score_col]
    temp['behavior_comment'] = df[comment_col]
    long_df = pd.concat([long_df, temp], ignore_index=True)

# Preview the final melted dataframe
long_df.head()


In [98]:
behavior_to_skill = {
    # Discovery / Research
    "research":                      "discovery",
    "high_impact_questions":         "discovery",
    "examine_challenges":            "discovery",

     # Drive Value
    "present_the_impact_story":      "drive_value",
    "share_key_insights":            "drive_value",
    "align_unique_differentiators":  "drive_value",
    
    # Deliver Business Case
    "deliver_compelling_proposals":  "deliver_biz_case",
    "prove_roi":    "deliver_biz_case",
    "handle_objections":             "deliver_biz_case",

    # Gain Commitment    
    "navigate_decision_making_processes": "gain_commitment",
    "set_purposeful_milestones":     "gain_commitment",
    "establish_alignment_accountability": "gain_commitment",
}

# ── 2.  Apply the mapping with `map` (fast & readable) ────────────────
long_df["skill"] = (
    long_df["behavior"]                 # existing column
           .str.lower()                 # normalise case
           .map(behavior_to_skill)      # look-up in dict
           .fillna("Other / Unmapped")  # fallback label
)

In [99]:
long_df

# BIC calculation

In [100]:
long_df['score'].unique()

In [101]:
valid_scores = ['1.0', '2.0', '3.0', '4.0', '5.0', 1, 2, 3, 4, 5]

df = long_df[long_df['score'].isin(valid_scores)] 

In [102]:
# Ensure 'date' and 'score' are in the right formats
df['calldate'] = pd.to_datetime(df['calldate'])
#--df_filtered = df[(df['calldate'] >= '2025-01-01') & (df['calldate'] <= '2025-04-01')].copy() # & (df['rep_segment'] == 'ESG')].copy()
df_filtered = df[(df['calldate'] >= '2024-10-01') & (df['calldate'] <= '2024-12-31') & (df['rep_segment'] == 'ESG')].copy()
df_filtered['score'] = pd.to_numeric(df_filtered['score'], errors='coerce')

# High scores only
df_high_scores = df_filtered[df_filtered['score'] >= 4]

# Count high scores per callid and skill
high_score_counts = df_high_scores.groupby(['callid', 'skill']).size().reset_index(name='high_score_behaviors')


best_in_class = high_score_counts[high_score_counts['high_score_behaviors'] >= 2]

# Count of best in class calls per skill
best_in_class_counts = best_in_class.groupby('skill')['callid'].nunique()

# Total number of calls where each skill was evaluated
total_calls_per_skill = df_filtered.groupby('skill')['callid'].nunique()

# Final percentage
best_in_class_percent = (best_in_class_counts / total_calls_per_skill * 100).round(2)

# Display
print(best_in_class_percent)


In [104]:
df_filtered.head()

In [105]:
# Step 4: Flag skills that are Best in Class (2+ high scores)
best_in_class_skills = high_score_counts[high_score_counts['high_score_behaviors'] >= 2]

# Step 5: Count how many skills are Best in Class per call
bcr_skill_counts_per_call = (
    best_in_class_skills.groupby('callid')['skill']
    .nunique()
    .reset_index(name='num_bcr_skills')
)

# Step 6: Flag calls that are Best in Class for all 5 skills
bcr_skill_counts_per_call['is_best_in_class_call'] = (bcr_skill_counts_per_call['num_bcr_skills'] >= 4)

# Step 7: Calculate %
total_calls = df_filtered['callid'].nunique()
bcr_call_count = bcr_skill_counts_per_call['is_best_in_class_call'].sum()
bcr_call_percent = round((bcr_call_count / total_calls) * 100, 2)

# Result
print(f"% of Calls that are Best in Class for all 5 skills: {bcr_call_percent}%")

In [106]:
print(bcr_call_count)

# Creating Proficiency Curves

## Skill Proficiency No Smoothing at Skill Level

In [109]:

# Step 1: Group by rep + skill and get average score
rep_skill_avg = df_filtered.groupby(['emailaddress', 'skill'])['score'].mean().reset_index()

# Step 2: Round to nearest score bucket
rep_skill_avg['score_bucket'] = rep_skill_avg['score'].round().astype(int)

# Step 3: Count number of reps per score bucket per skill
score_dist = rep_skill_avg.groupby(['skill', 'score_bucket'])['emailaddress'].nunique().reset_index()
score_dist.rename(columns={'emailaddress': 'rep_count'}, inplace=True)

# Step 4: Normalize by total reps per skill to get %
total_reps_per_skill = rep_skill_avg.groupby('skill')['emailaddress'].nunique().reset_index()
total_reps_per_skill.rename(columns={'emailaddress': 'total_reps'}, inplace=True)
score_dist = score_dist.merge(total_reps_per_skill, on='skill')
score_dist['rep_percent'] = score_dist['rep_count'] / score_dist['total_reps'] * 100

# Step 5: Fill missing score buckets for each skill
full_range = pd.DataFrame({'score_bucket': range(1, 6)})
skills = score_dist['skill'].unique()
smoothed_data = []

for skill in skills:
    df_skill = score_dist[score_dist['skill'] == skill].merge(full_range, on='score_bucket', how='right')
    df_skill['skill'] = skill
    df_skill['rep_percent'] = df_skill['rep_percent'].fillna(0)
    smoothed_data.append(df_skill)

score_dist_full = pd.concat(smoothed_data)

## Skill Proficiency Smoothing at Skill Level

In [110]:
# Step 6: Plot with smoothing–
plt.figure(figsize=(10, 6))

palette = sns.color_palette("Blues", n_colors=4)
for i, skill in enumerate(skills):
    df_skill = score_dist_full[score_dist_full['skill'] == skill].sort_values('score_bucket')
    x = df_skill['score_bucket']
    y = df_skill['rep_percent']
    
    # Interpolation for smooth curve
    x_new = np.linspace(1, 5, 300)
    spline = make_interp_spline(x, y, k=3)
    y_smooth = spline(x_new)
    y_smooth = np.clip(y_smooth, 0, 100)  # Keep % within 0–100

    plt.plot(x_new, y_smooth, label=skill, color=palette[i])

# Format plot
plt.title('Smoothed Skill Proficiency Curves')
plt.xlabel('Scores')
plt.ylabel('% Proficiency/Rep')
plt.xticks([1, 2, 3, 4, 5])
plt.yticks(np.arange(0, 101, 20))
plt.ylim(0, 100)
plt.xlim(1,5)
plt.grid(axis='x', linestyle=':', alpha=0.5)
plt.legend(title='Skill', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

## Skill Proficiency Smoothing - At Behavior Level

In [112]:

# Step 1: Calculate the percentage of times an employee scored each specific score (1, 2, 3, 4, 5) by behavior and skill
score_percentages = (
    df_filtered.groupby(['emailaddress', 'score', 'behavior', 'skill'])  # Include 'skill' in the groupby
    .size()  # Count occurrences of each score for each employee, behavior, and skill
    .reset_index(name='score_count')  # Create a new column for the count of each score
)

# Calculate the total number of calls for each employee by behavior and skill
total_calls = (
    df_filtered.groupby(['emailaddress', 'behavior', 'skill'])  # Group by employee, behavior, and skill
    .size()
    .reset_index(name='total_calls')  # Create a column for the total number of calls per employee, behavior, and skill
)

# Merge the total calls with the score percentages
score_percentages = score_percentages.merge(total_calls, on=['emailaddress', 'behavior', 'skill'], how='left')

# Calculate the percentage of each score for each employee and behavior
score_percentages['score_percentage'] = (score_percentages['score_count'] / score_percentages['total_calls']) * 100

# Step 2: Ensure all scores (1, 2, 3, 4, 5) are included for each employee, behavior, and skill
# Create a cartesian product of all employee_ids, behaviors, skills, and scores (1, 2, 3, 4, 5)
all_combinations = pd.DataFrame(
    list(itertools.product(score_percentages['emailaddress'].unique(), df_filtered['behavior'].unique(), df_filtered['skill'].unique(), [1, 2, 3, 4, 5])),
    columns=['emailaddress', 'behavior', 'skill', 'score']
)

# Merge the all_combinations dataframe with the score_percentages
score_percentages_complete = all_combinations.merge(score_percentages, on=['emailaddress', 'behavior', 'skill', 'score'], how='left')

# If an employee didn't get a particular score for a behavior and skill, set its percentage to 0
score_percentages_complete['score_percentage'].fillna(0, inplace=True)

# Step 3: Average the percentages of each score across all employees, grouped by behavior and skill
average_score_percentages = (
    score_percentages_complete.groupby(['skill', 'behavior', 'score'])['score_percentage']
    .mean()
    .reset_index()
    .rename(columns={'score_percentage': 'average_score_percentage'})
)


# Remove rows with missing score_percentage (NaN values) and impute with 0
score_percentages_complete['score_percentage'].fillna(0, inplace=True)

# Loop over each unique skill
for skill in average_score_percentages['skill'].unique():
    # Filter data for the specific skill
    skill_data = average_score_percentages[average_score_percentages['skill'] == skill]
    
    # Get the unique behaviors for this specific skill
    valid_behaviors = skill_data['behavior'].unique()
    
    # Create the plot for this skill
    plt.figure(figsize=(10, 6))

    # Loop over each behavior for the current skill
    for behavior in valid_behaviors:
        # Filter data for the current behavior within the specific skill
        behavior_data = skill_data[skill_data['behavior'] == behavior]

        # Check if behavior_data is empty (skip if no data for this behavior)
        if behavior_data.empty:
            continue

        # Extract x and y values for the plot
        x = behavior_data['score']
        y = behavior_data['average_score_percentage']

        # Skip behaviors with a score_percentage of 0 (irrelevant behavior for the current skill)
        if (y == 0).all():
            continue

        # Smooth the curve
        x_smooth = np.linspace(1, 5, 300)

        if len(x.unique()) >= 3:  # spline needs at least 3 points
            spline = make_interp_spline(x, y, k=2)  # use k=2 for small sets
            y_smooth = spline(x_smooth)
            y_smooth = np.clip(y_smooth, 0, 100)  # make sure % stays in bounds
            plt.plot(x_smooth, y_smooth, label=behavior)
        else:
            # Fallback to raw points if not enough to smooth
            plt.plot(x, y, marker='o', label=behavior)

    # Set plot labels and title
    plt.title(f'Skill Proficiency Breakdown by Behavior: {skill}')
    plt.xlabel('Average Score (Rounded)')
    plt.ylabel('% of Reps')
    plt.xticks([1, 2, 3, 4, 5])
    plt.yticks(np.arange(0, 101, 20))
    plt.ylim(0, 100)
    plt.xlim(1, 5)

    # Add gridlines for clarity
    plt.grid(axis='x', linestyle=':', alpha=0.5)

    # Add a legend for behaviors
    plt.legend(title='Behavior', bbox_to_anchor=(1.05, 1), loc='upper left')

    # Ensure tight layout for better spacing
    plt.tight_layout()

    # Show the plot
    plt.show()


# Export Data to Excel

In [18]:
import base64
import os
from IPython.display import HTML
def generate_excel_download_link(df, filename="skill_proficiency_distribution.xlsx", title = "Download Excel file", max_no_rows=65530):
    '''
    Returns a download link for the report data to be shown in jupyter notebook.
    Capped at max_no_rows due to excel limitation of not allowing more than max_no_rows urls in a single spreadsheet.
    '''
    if len(df)>max_no_rows:
        raise NameError(f'Number of rows exceeded: {len(df)}')
    df.to_excel(filename, index=False) # Save file temporarily (take advantage of pandas excel writer)
    with open(filename, 'rb') as f: # Open file in binary to convert to base64
        f_binary = f.read()
        b64 = base64.encodebytes(f_binary)
        payload = b64.decode()
    os.remove(filename) # Cleanup file
    html = '<a download="{filename}" href="data:text;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

In [19]:
def create_download_links(df, filename="skill_proficiency_distribution.xlsx", title = "Download Excel file", max_no_rows=65530):
    '''
    Returns a download link for the report data to be shown in jupyter notebook.
    Splits the report into buckets when number of rows exceeds max_no_rows.
  
    Returns
    -------
    links: list
        To see in jupyter, run:
            > for link in create_download_links(df, filename="my_report.xlsx"):
            >     display(link)
    '''
    filename_stub = filename.replace(".xlsx","")
    links = []
    
    if len(df)>max_no_rows:
        chunks = list(chunker(df, max_no_rows))
        print(f'Number of rows exceeded, splitting report into {len(chunks)} files')
        for i,chunk in enumerate(chunks):
            links.append(generate_excel_download_link(chunk, filename=f'{filename_stub}_{i}.xlsx', title = f'{title} - Partition {i}', max_no_rows=max_no_rows))
    else:
        links.append(generate_excel_download_link(df, filename = filename, title = title, max_no_rows=max_no_rows))
    return links

In [20]:
score_dist

In [25]:
for link in create_download_links(score_dist['rep_percent']):
    display(link)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Data provided
skills = ['Present Impact Story', 'Share Key Insights', 'Align Unique Differentiators']
percentages = [
    [36, 9, 5, 38, 13, 0],  # Present Impact Story
    [32, 0, 7, 16, 40, 5],  # Share Key Insights
    [31, 2, 11, 25, 28, 2]  # Align Unique Differentiators
]

# Score categories (0-5) and corresponding colors based on the attached image
colors = ['#FFFFFF', '#DFF2D8', '#A6D285', '#49814B', '#000000', '#444444']  # '#444444' is the new color added

# Create the figure and axis
fig, ax = plt.subplots(figsize=(12, 6))

# Plot one horizontal bar per skill
for skill_idx, skill in enumerate(skills):
    start = 0  # Initialize starting point for stacking
    for score_idx, percentage in enumerate(percentages[skill_idx]):
        bar = ax.barh(
            skill,  # Y-axis label (skill)
            percentage,  # Width of the bar (percentage value for the score)
            left=start,  # Starting point for this segment
            color=colors[score_idx],  # Color for the current score category
            edgecolor='black',  # Optional: Add edges for better visual separation
            label=f'Score {score_idx}' if skill_idx == 0 else None  # Add legend only for the first skill
        )

        # Add percentage text label
        if percentage > 0:  # Only display text for non-zero percentages
            ax.text(
                start + percentage / 2,  # Center the text within the bar
                skill,  # Corresponding skill on Y-axis
                f'{percentage}%',  # Text to display
                va='center',  # Vertically center the text
                ha='center',  # Horizontally center the text
                fontsize=8  # Adjust font size if needed
            )
        start += percentage  # Update starting point for next segment

# Optional: Add legend
handles, labels = ax.get_legend_handles_labels()
by_label = dict(zip(labels, handles))  # Remove duplicate labels
ax.legend(by_label.values(), by_label.keys(), title='Scores', bbox_to_anchor=(1.05, 1), loc='upper left')

# Set chart title and labels
ax.set_title('Skill Distribution by Scoring Categories')
ax.set_xlabel('Percentage of calls')
ax.set_ylabel('Skills')

# Adjust layout and display
plt.tight_layout()
plt.show()